# WIN Hackathon 2 - Computational modelling of behaviour - Part 1

**Developers**

Jan Grohn (jan.grohn@psy.ox.ac.uk), Miriam Klein-Flügge (miriam.klein-flugge@psy.ox.ac.uk)  


# Introduction

Many experiments in learning and decision making require the estimation of values and how these change over time. Learning which stimuli and actions have higher value allows humans and other animals to select which stimuli to approach or avoid, and which actions to take.

Some of the most commonly used models of learning and reinforcement were first developed by **Bush and Mosteller** in the 1950s, and further elaborated on by **Rescorla and Wagner** in the 1970s.

They have since developed into the field of **Reinforcement Learning** in computer science.

*Our aims for part 1 are:*

1.  understand an experimental paradigm of learning and decision making
2.  get to know a simple reinforcement learning model that can be used to explain behaviour in this task
3.  understand how changing parameters in this model affects its behaviour
4.  compute the subjective value of choice options

Note  that  the  final  section  (marked  ***)  is  a  slightly  mathsy optional section.  It’s  not  essential  to understand  it  to  progress  with  the  later  parts  in  the  hackathon.  However,  it  provides  a theoretical foundation for the analyses that you’ll do in later practicals and can be a fun
exercise to do!

## Import libraries and set global parameters

In [ ]:
# check if we are running on colab
try:
    from google.colab import files
    _ON_COLAB = True
except:
    _ON_COLAB = False

if _ON_COLAB:
    # this allows us to make interactive figures
    from google.colab import output
    output.enable_custom_widget_manager()

    # load in some custom functions for this hackathon
    !rm -r *
    !git clone -b winhack https://github.com/jangrohn/ComputationalmodelingBlockPractical
    !cp -R ComputationalmodelingBlockPractical/part1/ part1
    !rm -rf ComputationalmodelingBlockPractical

    %pip install -r ./part1/requirements.txt

import numpy as np
rng = np.random.default_rng(12345)
from part1 import plot_schedule, plot_interactive_RL_model, visualise_utility_function, plot_RL_weights

The last couple of lines of code in the previous cell load in some custom functions we wrote for this hackathon, which we will be using throughout this session. If you're interested in examining the code further, you can find them at https://github.com/JanGrohn/ComputationalModelingBlockPractical or by clicking the folder icon in the sidebar to the left and look through the scripts there.

# Section 1: Running a simple experiment

You can try out the task we will be analysing in this hackathon at  https://jangrohn.github.io/volatility_study/. If you would like, play through the task as best as you can to get a feel for the study. At the beginning of the task, initially choose one of the two conditions randomly but if you have the time also restart the task and play the other condition afterwards. The task doesn't save any of your data, but you will have an option to download it once you're finished.

# Section 2: Coding a simple reinforcement learning model in Python, and understanding the effects of varying learning rates

Here are some questions that you might want to discuss with your neighbours. Thinking through these questions will help you develop an intuition for what we will model later on.

*How did you learn whether green or orange was more likely to be rewarded?*

*How did you weigh this up against the number of points available when making your choices?*

*What do you think was the difference between condition 1 and condition 2 in the task?*

*Why do you think we included points – rather than simply letting participants learn which option has a higher reward probability?*

Reinforcement  learning  models  provide  a  way  of  tracking  the  probabilities  of  different outcomes when different actions are taken.

In this task, one of the key problems is to estimate *the probability that green is to be rewarded* on each trial. (Note that this is the *same* as 1 minus the probability that blue will be rewarded – so we only need to track one probability).

We can learn this probability by calculating a *prediction error* on each trial:

$$
\underbrace{\delta_t}_\textrm{prediction error} = \underbrace{o_t}_\textrm{outcome} - \underbrace{p_t}_\textrm{model prediction} \tag{Equation 1}
$$

where $\delta_t$ is the prediction error on trial $t$, $o_t$ is the outcome (1 if green was rewarded, 0 if blue was rewarded) on that trial, and $p_t$ is the current estimate of the probability that green will be rewarded (this is called `probOpt1` in the Python code). Have a think about what a prediction error might look like on a trial that does give reward and on a trial where no reward is obtained. What sign does it take in each case?

We then use this prediction error to *update our expectation* of how likely it is that green will be rewarded in the future.

$$
\underbrace{p_{t+1}}_\textrm{new prediction} = \underbrace{p_t}_\textrm{old prediction} + \underbrace{\alpha \delta_t}_\textrm{scaled prediction error} \tag{Equation 2}
$$

where $\alpha$ is a parameter called the *learning  rate*, whose value is $0 < \alpha \leq 1$. In your head, simulate different scenarios of this equation with a large (1) or small (0.1) learning rate and a positive or negative prediction error. What does the update look like, how does it differ?

The learning rate sets the *speed* at which the model learns from previous experience. Let’s explore directly in Python what happens when we vary the learning rate.

## Learning rates, and how they affect probability learning

The next code cell sets up, and plots, a simple ‘reward schedule’ where the true probability of green being rewarded is fixed at 0.8. Read through this code and try to understand what it is doing.

Once you have read through the code, run it to produce the figure.

### Learning with a fixed schedule

In [ ]:
def generate_schedule(trueProbability, rng = rng):
  '''
  Returns if option 1 (1) or option 2 (0) is rewarded on a trial

    Parameters:
        trueProbability(float array): The probability with which option 1 is
          rewarded on each trial
        rng (numpy random number generator, defaults to rng)

    Returns:
        opt1rewarded(int array): 1 if option 1 is rewarded on a trial, 0 if
          option 2 is rewarded on a trial
  '''
  # We'll simulate whether opt 1 was rewarded on every trial. For each trial, we
  # first generate a uniformly distributed random number between 0 and 1.
  randomNumbers = rng.random(len(trueProbability))

  # The trial is rewarded if that number is smaller than trueProbability, and
  # unrewarded otherwise.
  opt1Rewarded = randomNumbers < trueProbability

  # We return the outcome of this comparison (which is either True or False for
  # each trial) an an integer (which is 0 or 1 for each trial).
  return opt1Rewarded.astype(int)

# this is the true probability that green is rewarded
fixedProb = 0.8

# this is the number of trials
nTrials = 200

# reward probability on each trial
trueProbability = np.ones(nTrials, dtype = float) * fixedProb

# generate outcomes on each trial
opt1Rewarded = generate_schedule(trueProbability)

# visualise which option was rewarded on each trial
plot_schedule(opt1Rewarded, trueProbability)

The dots are at 1 every time that green is rewarded, and at 0 every time that blue is rewarded. The black dotted line is the true probability of reward (which the subject doesn’t know in the experiment).

The function RL_model defined in the next cell takes as its input: whether green is rewarded on each trial (`opt1rewarded`), what the model’s learning rate $\alpha$ is set to, and what the starting probability on the first trial is ($p_1$). It tries to return `probOpt1`, the probability of green being rewarded, as its output. However, the final two equations in the function
haven’t been completed. Open the next cell and complete the missing lines of code.

Once you’ve done this, you should be able to run this cell without receiving any errors, and the figure should now have a red trace that approaches the true probability:

### Simulating the RL model

In [ ]:
def RL_model(opt1Rewarded, alpha, startingProb = 0.5):
  '''
  Returns how likely option 1 is rewarded on each trial.

    Parameters:
        opt1rewarded(bool array): True if option 1 is rewarded on a trial, False
          if option 2 is rewarded on a trial.
        alpha(float): fixed learning rate, greater than 0, less than/equal to 1
        startingProb(float): starting probability (defaults to 0.5).

    Returns:
        probOpt1(float array): how likely option 1 is rewarded on each trial
          according to the RL model.
  '''

  # check that alpha has been set appropriately
  assert alpha > 0, 'Learning rate (alpha) must be greater than 0'
  assert alpha <= 1,'Learning rate (alpha) must be less than or equal to 1'

  # check that startingProb has been set appropriately
  assert startingProb >= 0, 'startingProb must be greater or equal than 0'
  assert startingProb <= 1, 'startingProb must be less than or equal to 1'

  # calculate the number of trials
  nTrials = len(opt1Rewarded)

  # pre-create some vectors we're going to assign into
  probOpt1 = np.zeros(nTrials, dtype = float)
  delta    = np.zeros(nTrials, dtype = float)

  # set the first trial's prediction to be equal to the starting probability
  probOpt1[0] = startingProb

  # solution:
  for t in range(nTrials-1):
        delta[t] = opt1Rewarded[t] - probOpt1[t]
        probOpt1[t+1] = probOpt1[t] + alpha*delta[t]

  return probOpt1

# this defines the model's estimated pronbabilty on the very first trial
startingProb = 0.5

# this is the model's learning rate
alpha = 0.05

# run the RL model
probOpt1 = RL_model(opt1Rewarded, alpha, startingProb)

plot_schedule(opt1Rewarded, trueProbability, probOpt1)

Now  try  playing  around  with  the  three  parameters `trueProb` `startingProb`, and `alpha`. Particularly try to understand the effects of varying `alpha`. What are the advantages of having a low $\alpha$? What are the advantages of having a high $\alpha$? If you set $\alpha$ to 1, how does the model behave? You can change the value using the sliders below.


In [ ]:
plot_interactive_RL_model(opt1Rewarded, trueProbability, RL_model, generate_schedule)

## Using a reversal schedule
In the experiment you performed earlier, the reward probability isn’t fixed, but it reverses at  various  points  during  the  experiment  –  so  at  some  points  green  is  more  likely  to  be rewarded, but at other points blue is more likely to be rewarded.

Let’s now generate such a schedule during which reversals take
place.

### Regenerating the schedule and the RL model

In [ ]:
# now we use a schedule with some reversals
trueProbability = np.concatenate((np.ones(25, dtype = float)*0.25,
                                  np.ones(25, dtype = float)*0.75,
                                  np.ones(25, dtype = float)*0.25,
                                  np.ones(25, dtype = float)*0.75,
                                  np.ones(100, dtype = float)*0.25))

opt1Rewarded = generate_schedule(trueProbability)

plot_interactive_RL_model(opt1Rewarded, trueProbability, RL_model, generate_schedule, change_trueProb = False)

Change the parameters above to see whether your reinforcement learner can keep track of this changing probability.

In the above schedule, there are periods where the reward environment is volatile (it reverses quite frequently) and other periods where the reward environment is stable (it doesn't change very frequently). Our hypothesis  is  that  people  adjust  their  learning  rates  depending  upon  the  stability  of  the environment.

Let's assume that subjects want to get their estimated probability as close to the true probability as possible. Based upon what you learnt about varying $\alpha$, in which environment might it be helpful to have a lower $\alpha$? In which environment might it be helpful to have a higher $\alpha$?

The reasoning behind why you might need to have a different $\alpha$ in different situations is discussed more fully in the following paper:

Behrens, T. E. J., Woolrich, M. W., Walton, M. E., & Rushworth, M. F. S. (2007). Learning the  value  of  information  in  an  uncertain  world,  Nature  Neuroscience  10(9),  1214–1221. http://doi.org/10.1038/nn1954

The computational model above is applied to every trial of the experiment. So far, it can make predictions about how likely a reward is to be hidden behind option 1 or option 2. However, the model does not yet specify which option would be a better choice as it currently still ignores that there are different reward magnitudes (number of points) associated with each option on each trial.

## Integrating reward magnitude and probability: utility

We need to now specify how the model combines the two dimensions to derive an overall utility (also sometimes called 'value') for each option.

Can you think of at least two different ways according to which participants might combine reward probabilities and magnitudes to figure out which option is better? Which of the different ways you have suggested is better or worse? Maybe think back to how you played the task.

### Multiplicative utility

As an example for why you need to combine the two dimensions, imagine that you are almost 100% sure that you will receive a reward if you choose an option, but if the reward is very small (say 2 out of 100 possible points), you might nevertheless not want to pick the option. We formalize this in equation 3:

$$
\underbrace{u}_\textrm{utility} = \underbrace{m}_\textrm{reward magnitude} \times \underbrace{p}_\textrm{reward probabilty} \tag{Equation 3}
$$

Here, we define the ultity $u$ as the product of the reward probability $p$ and magnitude $m$. We can refer to this way of calculating utility as 'multiplicative utility'. We define and plot this utility function in the next code cell.

Complete the utility function in the next code cell using equation 3. Then run the cell. The 3d plot it produces can be rotated by holding and dragging your cursor.

In [ ]:
# define the utility function as per equation 3
def multiplicative_utility(mag, prob):
  return mag*prob

visualise_utility_function(multiplicative_utility)

Describe why the utility function in equation 3 produces utility of the shape shown in the above plot.

### Additive utility

As another example, imagine that you consider reward magnitude and probability separately from each other. This is formalised in equation 4:

$$
\underbrace{u}_\textrm{utility} = \overbrace{\omega}^\textrm{magnitude weight} \times \underbrace{m}_\textrm{reward magnitude} + \overbrace{(1-\omega)}^\textrm{probability weight} \times \underbrace{p}_\textrm{reward probabilty} \tag{Equation 4}
$$

Here, reward magintude and probability are added together, which is why we refer to this type of utility as 'additive utility'. We weight up the two terms of the sum by a weight $\omega$, which is $0 \leq \omega \leq 1$. To ensure that probability and magnitude are on the same scale, we assume that the magnitude has been normalised to be on a range from $0$ to $1$ (i.e., it has been divided by $100$).  

At the extreme, you might even consider only reward magnitude or only reward probaility when making your choices. This can be modelled by setting $\omega$ to which values?

Run the next code cell that visualises additive utility.

In [ ]:
# additive utility function as per equation 4
def additive_utility(mag, prob, omega):
  return (omega*(mag/100) + (1-omega)*prob)*100

visualise_utility_function(additive_utility, omega = True)

Change the value of omega using the slider to see how it affects the shape of the utility function. Describe why the utility function in equation 4 produces utility of the shape shown in the above plot.

What are the major differences in the shape of multiplicative and additive utility?

# ***Section 3: Understanding learning rates as weighted sums

The key insight here is that the learning rate, $\alpha$, determines how the currently estimated probability on the next trial, $p_{t+1}$, is influenced by the past history of trials. We can think about  this in another way. Look back at equations 1 and 2. $p_{t+1}$ depends upon the outcome of the
most recent trial, $o_t$, but also the last trial’s probability estimate, $p_t$. However, $p_t$ could itself be written in terms of the probability of the previous trial’s outcome, $p_{t-1}$, and $p_{t-1}$. In turn $p_{t-1}$ could
be written in terms of $o_{t-2}$ and $p_{t-2}$. And so on.

In short, it becomes possible, when you are at trial $T$, to think of $p_{T+1}$ as a weighted sum of all the previous outcomes that the model experienced:

$$
p_{T+1} = (1-\alpha)^T p_1 + \sum^T_{t=1} w_t o_t \tag{Equation 5}
$$

Where $p_1$ is the starting probability (this becomes less and less important as we get further away from it, as $(1-\alpha)^T$ shrinks to zero), and $w_t$ is the amount of weight given to the outcome on trial $t$ on the current trial:

$$
w_t = \alpha(1-\alpha)^{T-t} \tag{Equation 6}
$$

$T$ is the current trial number, and so $T-t$ indexes how many trials into the past we are looking. In the final exercise, we see if you can derive this equation, step- by-step.

What does $w_t$ actually look like? This is explored in the next cell, which plots equation 6:

In [ ]:
plot_RL_weights()

This plot is also discussed at the beginning of the following book chapter (which is available as a PDF online at http://www.princeton.edu/~ndaw/dt.pdf):

Daw, N. D. and Tobler, P. N. (2014) Value learning through Reinforcement: The Basics of Dopamine and Reinforcement Learning. Chapter 15, Neuroeconomics (2nd edition), edited by Glimcher, P. W. and Fehr, E.

Try varying $\alpha$ and replotting the weights to see how it affects the weight assigned to the history of outcomes.

Last but not least, a bit of maths. Starting with equations 1 and 2, see if
you can derive equation 6, by substituting terms from the previous trial's equation for $p_t$. (This is the hardest exercise we've asked you to do today. If you manage to complete it, well done! If not, don't worry we will reveal how to do it in session 2).